# Summary

Try rewriting generator that streams from static backends. Want to do stopword truncation upfront. Might also be able to make this MUCH simpler.

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path

from jabberwocky.config import C
from jabberwocky.openai_utils import load_prompt, load_openai_api_key, GPT, \
    query_gpt_huggingface, truncate_at_first_stop, stream_response
from jabberwocky.streaming import _stream_fake_generator, _stream_response
from jabberwocky.utils import containerize
from htools import *

In [6]:
cd_root()

Current directory: /Users/hmamin/jabberwocky


In [8]:
GPT.switch('huggingface')

Switching openai backend to "huggingface".


In [50]:
def stream_fake_generator_v2(response, stop, start_i=0, prompt_i=0,
                             subwords=True, **kwargs):
    texts, fulls = containerize(*response)
    for i, (text, full) in enumerate(zip(texts, fulls)):
        trunc_text = truncate_at_first_stop(text, stop, **kwargs)
        # Converting to list lets us edit the last finish_reason more easily.
        pairs = [
            list(pair) for pair in
            _stream_response(
                trunc_text,
                {**full,
                 'index': i + start_i,
                 'prompt_index': prompt_i,
                 'finish_reason': None},
                subwords=subwords)
        ]
        reason = 'dummy' if trunc_text == text else 'stop'
        pairs[-1][-1]['finish_reason'] = reason
        yield from pairs

In [43]:
prompt = """Answer the math question.

Question: What is 3+4?
Answer: 7

Question: {}
Answer:"""

Don't use GPT.query as usual because the whole point here is to get the result without all my builtin postprocessing.

In [33]:
stop = ['\n\nQuestion: ', '\nAnswer:']
res = query_gpt_huggingface(prompt.format('What is 5-2?'), max_tokens=15,
                            engine=0, 
                            stop=stop, 
                            n=2,
                            stream=False)

/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:267: UserWarning: query_gpt_huggingface received unused kwargs {'stream': False}.
  warnings.warn('query_gpt_huggingface received unused kwargs '


In [54]:
res[0].append('Fake completion with no stopwords.')
res[1].append({'generated_text': 'Fake completion with no stopwords.'})

In [34]:
res[0]

[' 3.\n\nQuestion: 2+4-3? 6\nAnswer',
 ' 8\n\nQuestion: What is 1+2?\nAnswer: 3']

In [19]:
truncate_at_first_stop(res[0][0], stop_phrases=stop)

' 5'

In [35]:
list(_stream_fake_generator(res))

[(' 3',
  {'generated_text': ' 3.\n\nQuestion: 2+4-3? 6\nAnswer',
   'index': 0,
   'prompt_index': 0,
   'finish_reason': None}),
 ('.',
  {'generated_text': ' 3.\n\nQuestion: 2+4-3? 6\nAnswer',
   'index': 0,
   'prompt_index': 0,
   'finish_reason': None}),
 ('\n',
  {'generated_text': ' 3.\n\nQuestion: 2+4-3? 6\nAnswer',
   'index': 0,
   'prompt_index': 0,
   'finish_reason': None}),
 ('\n',
  {'generated_text': ' 3.\n\nQuestion: 2+4-3? 6\nAnswer',
   'index': 0,
   'prompt_index': 0,
   'finish_reason': None}),
 ('Question',
  {'generated_text': ' 3.\n\nQuestion: 2+4-3? 6\nAnswer',
   'index': 0,
   'prompt_index': 0,
   'finish_reason': None}),
 (':',
  {'generated_text': ' 3.\n\nQuestion: 2+4-3? 6\nAnswer',
   'index': 0,
   'prompt_index': 0,
   'finish_reason': None}),
 (' 2',
  {'generated_text': ' 3.\n\nQuestion: 2+4-3? 6\nAnswer',
   'index': 0,
   'prompt_index': 0,
   'finish_reason': None}),
 ('+',
  {'generated_text': ' 3.\n\nQuestion: 2+4-3? 6\nAnswer',
   'index': 0,

In [56]:
for tok, full in stream_fake_generator_v2(res, stop):
    print(tok, full)

 3 {'generated_text': ' 3.\n\nQuestion: 2+4-3? 6\nAnswer', 'index': 0, 'prompt_index': 0, 'finish_reason': None}
. {'generated_text': ' 3.\n\nQuestion: 2+4-3? 6\nAnswer', 'index': 0, 'prompt_index': 0, 'finish_reason': 'stop'}
 8 {'generated_text': ' 8\n\nQuestion: What is 1+2?\nAnswer: 3', 'index': 1, 'prompt_index': 0, 'finish_reason': 'stop'}
Fake {'generated_text': 'Fake completion with no stopwords.', 'index': 2, 'prompt_index': 0, 'finish_reason': None}
 completion {'generated_text': 'Fake completion with no stopwords.', 'index': 2, 'prompt_index': 0, 'finish_reason': None}
 with {'generated_text': 'Fake completion with no stopwords.', 'index': 2, 'prompt_index': 0, 'finish_reason': None}
 no {'generated_text': 'Fake completion with no stopwords.', 'index': 2, 'prompt_index': 0, 'finish_reason': None}
 stopwords {'generated_text': 'Fake completion with no stopwords.', 'index': 2, 'prompt_index': 0, 'finish_reason': None}
. {'generated_text': 'Fake completion with no stopwords.', 